## ED22B052 Lab11_A Neural Networks Forward Propagation

#### Problem Statement: Implement the forward propagation for a two-hidden-layer network for m-samples and n-features, as we discussed in class. Initialize the weights randomly. Use the data from the previous labs, such as logistic regression. You can choose the number of neurons in the hidden layer and use the sigmoid activation function. Report the evaluation metrics for the network.  Also, use other non-linear activation functions like ReLU and Tanh. Report the loss using both MSE and Cross Entropy.

In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, log_loss

# Load the data
data = pd.read_csv('3D_printing_data.csv')

# Convert text labels into numbers
material_encoder = LabelEncoder()
pattern_encoder = LabelEncoder()
data['material'] = material_encoder.fit_transform(data['material'])
data['infill_pattern'] = pattern_encoder.fit_transform(data['infill_pattern'])

# Separate input features and the output we want to predict
features = data.drop(columns=['elongation'])  # Predicting 'elongation'
target = data['elongation'].values.reshape(-1, 1)

# Make all feature values similar in scale
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

# Simple Neural Network class
class SimpleNeuralNetwork:
    def __init__(self, input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, activation):
        np.random.seed(42)
        self.weights1 = np.random.randn(input_nodes, hidden1_nodes) * 0.01
        self.bias1 = np.zeros((1, hidden1_nodes))
        self.weights2 = np.random.randn(hidden1_nodes, hidden2_nodes) * 0.01
        self.bias2 = np.zeros((1, hidden2_nodes))
        self.weights3 = np.random.randn(hidden2_nodes, output_nodes) * 0.01
        self.bias3 = np.zeros((1, output_nodes))
        self.activation = activation
        
    def forward_pass(self, inputs):
        self.layer1 = np.dot(inputs, self.weights1) + self.bias1
        self.activation1 = self.activation(self.layer1)
        
        self.layer2 = np.dot(self.activation1, self.weights2) + self.bias2
        self.activation2 = self.activation(self.layer2)
        
        self.output_layer = np.dot(self.activation2, self.weights3) + self.bias3
        return self.output_layer
    
    def predict(self, inputs):
        return self.forward_pass(inputs)

# Check how good the model is
def check_model_performance(model, inputs, real_values):
    predictions = model.predict(inputs)
    
    mse_score = mean_squared_error(real_values, predictions)
    
    # For binary classification version (used in log loss)
    binary_real = (real_values > np.median(real_values)).astype(int)
    prediction_probs = sigmoid(predictions)
    
    log_loss_score = log_loss(binary_real, prediction_probs)
    
    return mse_score, log_loss_score

# Set up the network
input_nodes = x_train.shape[1]
hidden1_nodes = 10
hidden2_nodes = 5
output_nodes = 1

# Try different activation functions
networks = {
    'Sigmoid': SimpleNeuralNetwork(input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, sigmoid),
    'ReLU': SimpleNeuralNetwork(input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, relu),
    'Tanh': SimpleNeuralNetwork(input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, tanh)
}

# Test each network
results = {}
for name, network in networks.items():
    network.forward_pass(x_train)
    mse_score, log_loss_score = check_model_performance(network, x_test, y_test)
    results[name] = {'MSE': mse_score, 'Cross Entropy': log_loss_score}

# Show the results
print("Neural Network Evaluation:")
print("=" * 40)
for name, scores in results.items():
    print(f"Using: {name} Activation")
    print(f"Mean Squared Error: {scores['MSE']:.4f}")
    print(f"Cross Entropy: {scores['Cross Entropy']:.4f}")
    print("-" * 40)


Neural Network Evaluation:
Using: Sigmoid Activation
Mean Squared Error: 3.6749
Cross Entropy: 0.6937
----------------------------------------
Using: ReLU Activation
Mean Squared Error: 3.6940
Cross Entropy: 0.6931
----------------------------------------
Using: Tanh Activation
Mean Squared Error: 3.6940
Cross Entropy: 0.6931
----------------------------------------
